In [1]:
import numpy as np
import glob
import pandas as pd
from PIL import Image
import cv2
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

In [ ]:
PATH2DATASET = '/mnt/SSD/workspace/atomic_hack/Сварные_швы/dataset/'

In [2]:
images = glob.glob(f'{PATH2DATASET}/*jpg')

In [4]:
idx2label = {
    0: 'прилегающие дефекты', 
    1: 'дефекты целостности',
    2: 'дефекты геометрии',
    3: 'дефекты постобработки',
    4: 'дефекты невыполнения'
}

In [5]:
df = pd.DataFrame(columns=['fname', 'x_center', 'y_center', 'width', 'height', 'x1', 'y1', 'x2', 'y2', 'label_idx', 'label'])
df

,fname,x_center,y_center,width,height,x1,y1,x2,y2,label_idx,label


In [6]:
for image_path in tqdm(images):
    fname = os.path.basename(image_path)
    img = cv2.imread(image_path)
    h, w, _ = img.shape
    label_path = image_path.replace('jpg', 'txt')
    
    with open(label_path) as f:
        labels = f.readlines()
    
    for label_data in labels:
        label_idx, x_center, y_center, width, height = [float(i) for i in label_data.split()]
        label_idx = int(label_idx)
        label = idx2label[label_idx]
        x1 = x_center - (width / 2)
        y1 = y_center - (height / 2)
        x2 = x1 + width
        y2 = y1 + height

        x1 = int(x1*w)
        x2 = int(x2*w)    
        y1 = int(y1*h)    
        y2 = int(y2*h)  
        
        df.loc[len(df)] = [fname, x_center, y_center, width, height, x1, y1, x2, y2, label_idx, label]

100%|███████████████████████████████████████| 1162/1162 [01:24<00:00, 13.77it/s]


In [12]:
df['group'] = df['fname'].apply(lambda x: os.path.basename(x).split()[0])
# df['fname'] = df['fname'].apply(lambda x: '/mnt/SSD/workspace/atomic_hack/Сварные_швы/dataset/'+x)

In [14]:
df['frame'] = df['fname'].apply(lambda x: int(os.path.basename(x).split()[1].split(')')[0].replace('(', '')))

In [12]:
for i in df['group'].unique():
    print(f'group = {i}', df[df['group']==i].value_counts('label_idx'))
    print()

group = 8 label_idx
2    183
3     72
Name: count, dtype: int64

group = 11 label_idx
2    84
Name: count, dtype: int64

group = 6 label_idx
2    97
3    19
1    11
Name: count, dtype: int64

group = 4 label_idx
0    351
2    124
1     56
3     45
4     26
Name: count, dtype: int64

group = 5 label_idx
0    393
2    198
1     72
4     51
3     48
Name: count, dtype: int64

group = 7 label_idx
0    568
2    299
1    134
4    104
3     68
Name: count, dtype: int64

group = 9 label_idx
2    53
3    26
1    19
Name: count, dtype: int64

group = 12 label_idx
1    94
4    14
Name: count, dtype: int64

group = 2 label_idx
0    62
Name: count, dtype: int64

group = 3 label_idx
0    21
2    18
4    10
3     9
Name: count, dtype: int64

group = 1 label_idx
0    87
2     7
1     2
Name: count, dtype: int64



In [22]:
val_df = df[df['group']=='5']

In [23]:
val_df = val_df[(val_df['frame']>=107) & (val_df['frame']<=223)]

In [24]:
val_df.value_counts('label_idx')

label_idx
0    286
2    118
4     40
1     35
3     22
Name: count, dtype: int64

In [28]:
df['split'] = 'train'

In [29]:
df.loc[val_df.index, 'split'] = 'val'

In [30]:
df.value_counts('split')

split
train    2924
val       501
Name: count, dtype: int64

In [31]:
df.to_csv('/mnt/SSD/workspace/atomic_hack/df.csv', index=False)